# Import 

In [1]:
import sys,os,copy

import open3d as o3d
import numpy as np
import trimesh

import matplotlib
import matplotlib.pyplot as plt

from sklearn.neighbors import KDTree

# for fast sparse matrix computation
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import inv

# for colormap
import matplotlib.cm as cm
from matplotlib.colors import Normalize

%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

# Load Data

In [2]:
# root directory for ShapeNetSem dataset
shapenet_root = "shapenetsem"
# directory where we save our new dataset
data_root = "shapenet5k"

In [3]:
def get_rotation(alpha,beta,gamma):
    s_alpha = np.sin(alpha)
    c_alpha = np.cos(alpha)
    s_beta = np.sin(beta)
    c_beta = np.cos(beta)
    s_gamma = np.sin(gamma)
    c_gamma = np.cos(gamma)
    R = np.array([
        [c_alpha*c_beta, c_alpha*s_beta*s_gamma - s_alpha*c_gamma, c_alpha*s_beta*c_gamma + s_alpha*s_gamma],
        [s_alpha*c_beta, s_alpha*s_beta*s_gamma + c_alpha*c_gamma, s_alpha*s_beta*c_gamma - c_alpha*s_gamma],
        [-s_beta, c_beta*s_gamma, c_beta*c_gamma]
    ])

    return R.T

In [4]:
# create an empty directory
!mkdir shapenet5k

In [ ]:
# get OBB for all files
obb_dict = {}
filenames = list(os.listdir(shapenet_root))
i = 0
for filename in filenames:
    if i<1000 and filename.endswith(".obj"):
        # read the mesh
        mesh = o3d.io.read_triangle_mesh(os.path.join(shapenet_root,filename))
        # sample points
        pcd = mesh.sample_points_uniformly(number_of_points = 5000)
        # center
        origin = np.zeros((3,1))
        pcd.translate(origin,relative=False)
        # scale
        pts = np.asarray(pcd.points)
        dist = np.max(np.sqrt(np.sum(pts**2, axis = 1)),0)
        pcd.scale(1/dist,center=origin)

        # generate a rotation matrix with uniformly sampled angles
        [alpha, beta, gamma] = np.random.uniform(0,np.pi, size = (3))
        R = get_rotation(alpha, beta, gamma)
        # rotate point cloud
        pcd = pcd.rotate(R)
        try:
            # compute the OBB
            obb = pcd.get_oriented_bounding_box()
            # append into the dataset
            i += 1
            new_filename = "pcd_{}.xyz".format(i)
            obb_dict[new_filename] = {
                'center':obb.center.flatten().tolist(),
                'extent':obb.extent.flatten().tolist(),
                'R':obb.R.flatten().around().tolist()
            }
            # also save the rotated point clouds
            o3d.io.write_point_cloud(os.path.join(data_root,new_filename), pcd)
        except:
            print('Skipped',filename)

In [95]:
# split into train and test
filenames = list(obb_dict.keys())
n = len(filenames)
indices = list(range(n))
random.shuffle(indices)
# train_set
train_indices = indices[:700]
train_labels = {filenames[i]:obb_dict[filenames[i]] for i in train_indices}
# test set
test_indices = indices[-300:]
test_labels = {filenames[i]:obb_dict[filenames[i]] for i in test_indices}

In [75]:
# save the ground truth OBB vertices
import json

with open('shapenet_labels.json','w') as fp:
    json.dump(obb_dict,fp)

In [98]:
# save train and test labels
with open('shapenet_train_labels.json','w') as fp:
    json.dump(train_labels,fp)

with open('shapenet_test_labels.json','w') as fp:
    json.dump(test_labels,fp)

# Visualization

In [104]:
with open("shapenet_train_labels.json", 'r') as fp:
    obb_dict = json.load(fp)

In [100]:
filenames = list(obb_dict.keys())

In [101]:
def visualize(filename,data_root):
    filepath = os.path.join(data_root,filename)
    pcd = o3d.io.read_point_cloud(filepath)
    obb_params = obb_dict[filename]
    center = np.array(obb_params['center'],dtype=np.float64).reshape(3,1)
    extent = np.array(obb_params['extent'],dtype=np.float64).reshape(3,1)
    R = np.array(obb_params['R'],dtype=np.float64).reshape(3,3)
    obb = o3d.geometry.OrientedBoundingBox(center=center,extent=extent,R=R)
    obb_line_set = o3d.geometry.LineSet.create_from_oriented_bounding_box(obb)
    obb_line_set.paint_uniform_color([1,0,0])
    aab = pcd.get_axis_aligned_bounding_box()
    aab_line_set = o3d.geometry.LineSet.create_from_axis_aligned_bounding_box(aab)
    o3d.visualization.draw_geometries([pcd,obb_line_set,aab_line_set])

In [ ]:
visualize(filenames[100],'shapenet5k')

In [ ]:
visualize(filenames[90],'shapenet5k')